In [1]:
from probe_experiment import *
from handcrafted_datasets import Dataset
from models import *
import torch
import torch.nn as nn
from sklearn.metrics import roc_curve, roc_auc_score

In [2]:
dataset = Dataset('walls', 'C:\\Users\\andre\\go-ai\\data', 0.8, 0.2, 1024)

def get_train_loader():
    dataset.shuffle('train')
    return dataset.loader('train', max_ram_files=25)

def get_test_loader():
    return dataset.loader('test', max_ram_files=25)

done loading data
split sizes:
train 552
test 138


In [3]:
def flatten_bce_loss(pred, target):
    return torch.nn.BCEWithLogitsLoss()(pred, nn.Flatten()(target))
criterion = flatten_bce_loss

In [4]:
go_model = load_go_model_from_ckpt('model_ckpt.pth.tar', rm_prefix=True)
feat_models = [CutModel(go_model, i).cuda() for i in range(8)]
n_channels = [8, 64, 64, 64, 48, 48, 32, 32]
probe_models = [nn.Sequential(nn.Conv2d(nc, 1, 19, padding=9), nn.Flatten()).cuda() for nc in n_channels]

In [ ]:
aucs = []
for depth in range(6, 8):
    print('probing at depth %d' % depth)
    feat_model = feat_models[depth]
    probe_model = probe_models[depth]
    exp = ProbeExperiment(get_train_loader, get_test_loader, feat_model)
    config = {'name':'walls/wall_19x19cnn_d%d'%depth, 'write_log':True, 'progress_bar':True, 'save_ckpt':False}
    optimizer = torch.optim.Adam(probe_model.parameters())
    exp.run(probe_model, criterion, optimizer, 5, config)
    preds, labels = exp.get_predictions(probe_model, get_test_loader())
    auc = roc_auc_score(labels.flatten(), preds.flatten())
    aucs.append(auc)
    print(auc)

probing at depth 6


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 552/552 [01:42<00:00,  5.41it/s]


[LOG] epoch 0 loss 0.120991, new best


  0%|                                                                                                                                    | 0/552 [00:00<?, ?it/s]

[LOG] epoch 1 loss 0.113063, new best


...

[LOG] epoch 2 loss 0.117639, new best


  0%|                                                                                                                                    | 0/552 [00:00<?, ?it/s]

[LOG] epoch 3 loss 0.114092, new best


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 552/552 [01:58<00:00,  4.64it/s]


[LOG] epoch 4 loss 0.111580, new best
